In [16]:
def sample(MDP, Pi, timestep_max, number):
    ''' 采样函数,策略Pi,限制最长时间步timestep_max,总共采样序列数number '''
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)]  # 随机选择一个除s5以外的状态s作为起点
        # 当前状态为终止状态或者时间步太长时,一次采样结束
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            rand, temp = np.random.rand(), 0   ## 初始化概率，以及动作概率是0
            # 在状态s下根据策略选择动作
            for a_opt in A:       ## 遍历每个动作
                temp += Pi.get(join(s, a_opt), 0)    ## 拿到状态+动作的名称，然后从策略Pi_1内拿到对应的动作概率
                if temp > rand:   ## 该状态和动作对应的概率，满足概率条件，则执行动作action
                    a = a_opt        ## 执行动作的action，a是动作名称
                    r = R.get(join(s, a), 0)    ## 拿到状态+动作的名称，然后从状态+动作的奖励 R 内拿到对应的动作奖励
                    break                       ## 已经执行了动作，退出动作的循环
            rand, temp = np.random.rand(), 0    ## 初始化概率，以及状态的概率
            # 根据状态转移概率得到下一个状态s_next
            for s_opt in S:   ## 遍历状态列表
                temp += P.get(join(join(s, a), s_opt), 0)  ## 组合了状态+动作+转移的状态名称，然后从状态+动作+转移状态的概率 P 内拿到对应的转移概率
                if temp > rand:      ##    满足概率条件，则执行状态转移
                    s_next = s_opt   ##    转移到的状态是 s_next
                    break            ##    已经转移了状态，退出状态的循环
            ## s是当前的状态，a是执行的动作，r是执行动作的奖励，s_next是执行动作以后转移到的状态
            episode.append((s, a, r, s_next))  # 把（s,a,r,s_next）元组放入序列中
            s = s_next  # s_next变成当前状态,开始接下来的循环
        episodes.append(episode)
    return episodes


# 采样5次,每个序列最长不超过1000步
episodes = sample(MDP, Pi_1, 20, 5)
print('第一条序列\n', episodes[0])
print('第二条序列\n', episodes[1])
print('第五条序列\n', episodes[4])

# 第一条序列
#  [('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s5', 0, 's5')]
# 第二条序列
#  [('s4', '概率前往', 1, 's4'), ('s4', '前往s5', 10, 's5')]
# 第五条序列
#  [('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]

第一条序列
 [('s4', '前往s5', 10, 's5')]
第二条序列
 [('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's3'), ('s3', '前往s5', 0, 's5')]
第五条序列
 [('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]


In [26]:
# 对所有采样序列计算所有状态的价值
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):  #一个序列从后往前计算
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]


timestep_max = 20
# 采样1000次,可以自行修改
episodes = sample(MDP, Pi_1, timestep_max, 10000)
gamma = 0.5
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma)
print("使用蒙特卡洛方法计算MDP的状态价值为\n", V)

# 使用蒙特卡洛方法计算MDP的状态价值为
#  {'s1': -1.228923788722258, 's2': -1.6955696284402704, 's3': 0.4823809701532294,
# 's4': 5.967514743019431, 's5': 0}

使用蒙特卡洛方法计算MDP的状态价值为
 {'s1': -1.2215579604515683, 's2': -1.6773800314386655, 's3': 0.5157195420986501, 's4': 6.061523470412039, 's5': 0}


In [30]:
def occupancy(episodes, s, a, timestep_max, gamma):
    ''' 计算状态动作对（s,a）出现的频率,以此来估算策略的占用度量 '''
    rho = 0
    total_times = np.zeros(timestep_max)  # 记录每个时间步t各被经历过几次
    occur_times = np.zeros(timestep_max)  # 记录(s_t,a_t)=(s,a)的次数
    for episode in episodes:              ## 遍历每个采样的内容
        for i in range(len(episode)):     ## 遍历每个采样内部的状态、动作的内容
            (s_opt, a_opt, r, s_next) = episode[i]   ## 拿到每个采样内的 状态、动作、动作的奖励、下一个状态
            total_times[i] += 1                      ## 统计序列所在index的次数
            if s == s_opt and a == a_opt:            ## 状态、动作和给定的相同
                occur_times[i] += 1                  ## 也就是（状态，动作）对的次数+1
    for i in reversed(range(timestep_max)):       ## 逆序算占用度量
        if total_times[i]:                        ## 序列所在的index有值
            rho += gamma**i * occur_times[i] / total_times[i]        ## 按照公式运算
    return (1 - gamma) * rho


gamma = 0.5
timestep_max = 1000

episodes_1 = sample(MDP, Pi_1, timestep_max, 10000)  ## 采样，参数是MDP，Pi_1是策略，timestep_max是最长时间步，10000是采样的次数
episodes_2 = sample(MDP, Pi_2, timestep_max, 10000)  ## 和上面只有策略不同，Pi_2
rho_1 = occupancy(episodes_1, "s4", "概率前往", timestep_max, gamma)    ## 算占用度量
rho_2 = occupancy(episodes_2, "s4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)

# 0.112567796310472 0.23199480615618912

0.11712105221274086 0.2329241059970849
